In [1]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
mnist = tf.keras.datasets.mnist

In [2]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [4]:
q_aware_model = tfmot.quantization.keras.quantize_model(model)

In [5]:
q_aware_model.compile(optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])

q_aware_model.fit(x_train, y_train, epochs=5)
q_aware_model.evaluate(x_test, y_test)
q_aware_model.summary()


Epoch 1/5


1875/1875 [==============================] - 3s 1ms/step - loss: 3.1005 - accuracy: 0.1782
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 88.8932 - accuracy: 0.2972
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 107.7350 - accuracy: 0.8558
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 26.7157 - accuracy: 0.9078
Epoch 5/5
313/313 [==============================] - 0s 792us/step - loss: 6.6073 - accuracy: 0.9460
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLa  (None, 28, 28)            3         
 yer)                                                            
                                                                 
 quant_flatten (QuantizeWra  (None, 784)               1         
 pperV2)                                                         
      

In [6]:
q_aware_model.predict(x_test[0])

ValueError: in user code:

    File "f:\FPGA\GUSN\.venv\Lib\site-packages\tf_keras\src\engine\training.py", line 2436, in predict_function  *
        return step_function(self, iterator)
    File "f:\FPGA\GUSN\.venv\Lib\site-packages\tf_keras\src\engine\training.py", line 2421, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "f:\FPGA\GUSN\.venv\Lib\site-packages\tf_keras\src\engine\training.py", line 2409, in run_step  **
        outputs = model.predict_step(data)
    File "f:\FPGA\GUSN\.venv\Lib\site-packages\tf_keras\src\engine\training.py", line 2377, in predict_step
        return self(x, training=False)
    File "f:\FPGA\GUSN\.venv\Lib\site-packages\tf_keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\krzys\AppData\Local\Temp\__autograph_generated_fileaxmok7em.py", line 75, in tf__call
        ag__.if_stmt('training' in ag__.ld(args), if_body_

In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\krzys\AppData\Local\Temp\tmp_koa0qsk\assets


INFO:tensorflow:Assets written to: C:\Users\krzys\AppData\Local\Temp\tmp_koa0qsk\assets
f:\FPGA\GUSN\.venv\Lib\site-packages\tensorflow\lite\python\convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [13]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
signatures = interpreter.get_signature_list()
signatures

f:\FPGA\GUSN\.venv\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


{'serving_default': {'inputs': ['flatten_input'],
  'outputs': ['quant_dense_1']}}